# Gathering Data from https://www.altafsir.com/

In [1]:
import pandas as pd
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

In [2]:
#Install Driver
driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/2x/qhy93fs134s_883czg81ywrm0000gn/T/ipykernel_89072/2409737876.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## Verse Translation

In [3]:
for j in (21,17,10,8,7,5,4,3):
    work = []
    url = 'https://www.altafsir.com/ViewTranslations.asp?Display=yes&SoraNo=1&Ayah=0&toAyah=0&Language=2&LanguageID=2&TranslationBook='+str(j)
    driver.get(url)
    select = Select(driver.find_element('id','TranslationBook'))
    selected_option = select.first_selected_option
    print(selected_option.text)
    work.append(selected_option.text)
    
    for i in range(1, 115):
        url = 'https://www.altafsir.com/ViewTranslations.asp?Display=yes&SoraNo=' + str(i) + '&Ayah=0&toAyah=0&Language=2&LanguageID=2&TranslationBook='+str(j)
        driver.get(url)
        
        driver.switch_to.frame(driver.find_element(By.TAG_NAME, "iframe"))
        work.append(driver.find_element(By.TAG_NAME, "body").text)
        #print(i)
    
    clean_surah = []
    for i in work:
        surah = i.split("\n")
        ayats = ""
        for i in range(len(surah)):
            if i == 0:
                #print(surah[i])
                ayats += surah[i] + "\n"
            elif i == 1:
                continue
            else:
                if len(surah[i]) > 0:
                    if surah[i][0] == '[':
                        continue
                    else:
                        #print(surah[i])
                        ayats += surah[i] + "\n"
        ayats = ayats[:-1]
        clean_surah.append(ayats.split("\n"))

    surah_s = []
    ayat_s = []
    name_s = []
    translation = []
    worker=work[0].replace(" ", "_")

    surah_num = 0
    for i in clean_surah:
        ayat_num = 1
        for k in range(1, len(i)):
            name_s.append(i[0])
            surah_s.append(surah_num)
            ayat_s.append(ayat_num)
            translation.append(i[k])
            #print(worker, name_s, surah_s, ayat_s)
            ayat_num += 1
        surah_num+=1
    df = pd.DataFrame(data=zip(name_s, surah_s, ayat_s, translation), columns=['Name', 'Surah', 'Ayat', 'Verse'])
    df.to_csv(worker + "_translation.csv", index=False)
    
    

Muhammad Tahir-ul-Qadri
Recitations of Ibrahim walk from saheeh international
Royal Aal al-Bayt Institute Translation
Martin Lings
Muhammad Asad
Arthur J. Arberry
Abdullah Yusuf Ali
Marmaduke Pickthall


# Tafaseer

In [4]:
TAFSEERS = []
  
# 74, 73, 107, 108 completed
for k in (107, 108):
    url = 'https://www.altafsir.com/Tafasir.asp?tMadhNo=0&tTafsirNo=' + str(k) + '&tSoraNo=1&tAyahNo=1&tDisplay=yes&UserProfile=0&LanguageId=2'
    driver.get(url)    
    worker = []
    select = Select(driver.find_element('id','Tafsir'))
    selected_option = select.first_selected_option
    print(selected_option.text)
    worker.append(selected_option.text)
    
    for i in range(1, 115):
        url = 'https://www.altafsir.com/Tafasir.asp?tMadhNo=0&tTafsirNo=' + str(k) + '&tSoraNo='+str(i)+'&tAyahNo=1&tDisplay=yes&UserProfile=0&LanguageId=2'
        driver.get(url)
        select = Select(driver.find_element('id','Ayat'))
        ayats = len(select.options)
        #print("Ayats", ayats)
        for j in range(1, ayats + 1):
            url = 'https://www.altafsir.com/Tafasir.asp?tMadhNo=0&tTafsirNo=' + str(k) + '&tSoraNo='+str(i)+'&tAyahNo='+str(j)+'&tDisplay=yes&UserProfile=0&LanguageId=2'
            driver.get(url)
            try:
                worker.append(driver.find_element(By.CLASS_NAME, "TextAyah").text)
                worker.append(driver.find_element(By.CLASS_NAME, 'TextResultEnglish').text)
            except:
                worker.append("")
                worker.append("")
            
        #print(i)
    
    # ONLY FOR 1 Worker
    worker_name = worker[0].replace(" ", "_")
    arabic = []
    tafsir = []
    for i in range(1, len(worker)):
        if i%2==0:
            tafsir.append(worker[i])
        else:
            arabic.append(worker[i].replace("{","").replace("}", "").strip())
    #print(worker)
    df = pd.DataFrame(data=zip(arabic, tafsir), columns=['Arabic', 'Tafseer'])
    df.head()
    df.to_csv(worker_name + "_tafseer.csv", index=False)

Kashani Tafsir
Al Qushairi Tafsir


# Took 4 hours to run per Worker